In [11]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [12]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random

In [13]:
import mylibrary.datasets as datasets

In [14]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [15]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

train_size = len(train_label_)

In [16]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [17]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)

In [18]:
input_size = 784
output_size = 10

In [19]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [20]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [21]:
learning_rate = 0.0003
batch_size = 50

In [22]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

## Making of matrix factorized layer

In [23]:
class PairBilinear(nn.Module):
    def __init__(self, dim, grid_width):
        super().__init__()
        self.dim = dim
        self.grid_width = grid_width
        
        self.num_pairs = self.dim // 2
        along_row = torch.linspace(0, 1, self.grid_width).reshape(1, -1)
        along_col = torch.linspace(0, 1, self.grid_width).reshape(-1, 1)
        self.Y = torch.stack([along_row+along_col*0, along_row*0+along_col])
        self.Y = torch.repeat_interleave(self.Y.unsqueeze(0), self.num_pairs, dim=0)
        self.Y = nn.Parameter(self.Y)
        
        self.pairW = torch.eye(2).unsqueeze(0).repeat_interleave(self.num_pairs, dim=0)*0.5
        self.pairW = nn.Parameter(self.pairW)
    
    def forward(self, x):
        bs = x.shape[0]
        
############# This block ########################
        ### this block is significantly faster
        x = x.view(bs, -1, 2).transpose(0,1)
        x = torch.bmm(x, self.pairW)
        x = x.transpose(1,0).reshape(-1, 2)
        
############# OR This block ########################
#         x = x.view(-1, 2)
#         _wi = torch.arange(self.num_pairs).repeat(bs)
#         _W = self.pairW[_wi]
#         x = torch.bmm(x.unsqueeze(1), _W).squeeze(1)
####################################################
        
        x = x*self.grid_width
        index = torch.clamp(x.data, 0, self.grid_width-2)
        index = torch.floor(index)
        x = x-index
        
        index = (index.repeat_interleave(2, dim=0))
        
        _bi = torch.arange(bs).repeat_interleave(self.num_pairs*2)
        _gi = torch.arange(self.num_pairs).repeat_interleave(2).repeat(bs)
        _pi = torch.tensor([0,1], dtype=torch.long).repeat(bs*self.num_pairs)

#         _xc, _yc = tuple(index.type(torch.long).t())
        index_ = index.long().t()
        _xc, _yc = index_[0], index_[1]

#         print("types", _gi.dtype, _pi.dtype, _xc.dtype, _yc.dtype)
        f00 = self.Y[_gi, _pi, _xc, _yc]
        f01 = self.Y[_gi, _pi, _xc, _yc+1]
        f10 = self.Y[_gi, _pi, _xc+1, _yc]
        f11 = self.Y[_gi, _pi, _xc+1, _yc+1]
        #### https://en.wikipedia.org/wiki/Bilinear_interpolation
        a00 = f00
        a01 = f01-f00
        a10 = f10-f00
        a11 = f11-f10-f01+f00
        
        ##### this doubles the multiplication for x,y
#         x = x.repeat_interleave(2, dim=0)
#         y = a00 + x[:, 0]*a10 + x[:, 1]*a01 + x[:, 0]*x[:, 1]*a11

        ##### this repeats in individual way
        _x, _y = x[:, 0].repeat_interleave(2), x[:, 1].repeat_interleave(2)
        y = a00 + _x*a10 + _y*a01 + _x*_y*a11

        ### now 
        y = y.view(bs, -1)
        return y

In [24]:
pairBL = PairBilinear(784, 10).to(device)

pairBL_s = torch.jit.script(pairBL)
_a = torch.randn(100, 784).to(device)

In [25]:
# pairBL(_a) 
pairBL_s(_a) 

tensor([[-0.4924,  0.5181, -0.1708,  ...,  1.1631,  0.3801, -0.6424],
        [-0.1357, -0.1246,  0.7318,  ..., -0.9442, -0.3542,  0.4738],
        [ 0.0874,  0.6953, -0.2740,  ...,  0.2748, -0.6529,  0.7291],
        ...,
        [-0.1709, -0.6662, -0.2480,  ...,  0.2030, -0.2794, -0.7709],
        [ 0.2020,  1.4538, -0.1864,  ..., -0.0386, -0.1961, -0.8701],
        [ 0.3333, -0.1944,  0.0835,  ..., -0.1766,  0.5230,  0.9935]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [26]:
# %timeit pairBL(_a) 

In [27]:
# %timeit pairBL_s(_a) 

In [28]:
pairBL.Y.shape

torch.Size([392, 2, 10, 10])

In [29]:
class BiasLayer(nn.Module):
    def __init__(self, dim, init_val=0):
        super().__init__()
        self.bias = nn.Parameter(torch.ones(dim)*init_val)
        
    def forward(self, x):
        return x+self.bias

In [30]:
class FactorizedPairBilinearSpline(nn.Module):
    
    def __init__(self, input_dim, grid_width):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.input_dim = input_dim
        num_layers = int(np.ceil(np.log2(input_dim)))
            
        self.facto_nets = []
        self.idx_revidx = []
        for i in range(num_layers):
            idrid = self.get_pair(self.input_dim, i+1)
            net = PairBilinear(self.input_dim, grid_width)
            self.facto_nets.append(net)
            self.idx_revidx.append(idrid)
        self.facto_nets = nn.ModuleList(self.facto_nets)
            
    def get_pair(self, inp_dim, step=1):
        dim = 2**int(np.ceil(np.log2(inp_dim)))
        assert isinstance(step, int), "Step must be integer"

        blocks = (2**step)
        range_ = dim//blocks
        adder_ = torch.arange(0, range_)*blocks

        pairs_ = torch.Tensor([0, blocks//2])
        repeat_ = torch.arange(0, blocks//2).reshape(-1,1)
        block_map = (pairs_+repeat_).reshape(-1)

        reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
        indx = reorder_for_pair.type(torch.long)
        indx = indx[indx<inp_dim]

        rev_indx = torch.argsort(indx)
        return indx, rev_indx
    
    def forward(self, x):
        ## swap first and then forward and reverse-swap
        y = x
        for i in range(len(self.facto_nets)):
            idx, revidx = self.idx_revidx[i]
            y = y[:, idx]
            y = self.facto_nets[i](y) 
            y = y[:, revidx]
        y = x + y ## this is residual addition... remove if only want feed forward
        return y

In [31]:
pfL = FactorizedPairBilinearSpline(100, 5)

In [32]:
pfL(torch.randn(2, 100))

tensor([[-5.7361e-01,  1.9113e+00,  5.5239e-02,  9.3295e-02, -4.9530e-02,
         -4.2595e-01, -2.0145e-01,  6.2913e-01,  8.1817e-01, -8.3959e-02,
         -2.3481e-01,  8.5498e-01,  3.6066e-01,  1.9123e-01,  5.2535e-01,
          1.3376e+00,  8.8672e-01,  1.8753e-01,  1.7154e+00, -1.0448e+00,
          2.3597e+00, -7.0860e-01,  1.0935e-01,  6.0261e-01,  9.2030e-01,
          1.3315e+00, -7.8289e-02, -4.2865e-01,  2.3912e+00,  9.7522e-01,
          3.6417e-02,  2.2369e-01, -2.6051e-01, -1.9527e-03,  3.1257e-01,
          4.2857e-01,  1.6218e+00,  2.1368e+00,  1.9134e-02, -1.5613e+00,
         -1.4937e+00, -1.0976e+00,  1.1936e+00,  3.1138e-02, -2.1533e+00,
         -1.6499e+00,  1.0311e+00,  4.7011e-01,  7.5851e-01,  1.1250e+00,
          1.1244e+00, -1.0350e+00, -7.0881e-01, -9.7736e-01, -1.9764e+00,
          9.3808e-01, -5.6322e-01,  4.7274e-01,  2.1782e-01,  7.8118e-01,
          2.4553e-01,  6.5575e-01, -7.1200e-01,  1.6548e+00,  7.7838e-01,
          1.1009e+00, -2.0472e+00, -6.

In [33]:
pfL.facto_nets

ModuleList(
  (0): PairBilinear()
  (1): PairBilinear()
  (2): PairBilinear()
  (3): PairBilinear()
  (4): PairBilinear()
  (5): PairBilinear()
  (6): PairBilinear()
)

In [34]:
param_count = sum([torch.numel(p) for p in pfL.parameters()])
param_count

18900

In [35]:
class FactorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bias = BiasLayer(784)
        self.la1 = FactorizedPairBilinearSpline(784, grid_width=5)
        self.bn1 = nn.BatchNorm1d(784)
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bias(x)
        x = self.la1(x)
#         x = self.bn1(x)
        x = torch.relu(x)
        x = self.fc(x)
        return x

In [36]:
# class FactorNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.bias = BiasLayer(784)
#         self.la1 = FactorizedPairBilinearSpline(784, grid_width=5)
#         self.bn1 = nn.BatchNorm1d(784)
#         self.la2 = FactorizedPairBilinearSpline(784, grid_width=5)
#         self.bn2 = nn.BatchNorm1d(784)
#         self.fc = nn.Linear(784, 10)
        
#     def forward(self, x):
#         x = self.bias(x)
#         x = self.bn1(self.la1(x))
#         x = torch.relu(x)
#         x = self.bn2(self.la2(x))
#         x = torch.relu(x)
#         x = self.fc(x)
#         return x

In [37]:
class OrdinaryNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = nn.Linear(784, 784, bias=False)
        self.bn1 = nn.BatchNorm1d(784)
        self.la2 = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.la2(x)
        return x

In [38]:
model = FactorNet()
param_count = sum([torch.numel(p) for p in model.parameters()])
param_count

221882

In [39]:
model = OrdinaryNet()
param_count1 = sum([torch.numel(p) for p in model.parameters()])
param_count1, param_count1/param_count

(624074, 2.812639150539476)

### Model Development

In [44]:
torch.manual_seed(0)
model = FactorNet().to(device)
# model = OrdinaryNet().to(device)
model

FactorNet(
  (bias): BiasLayer()
  (la1): FactorizedPairBilinearSpline(
    (facto_nets): ModuleList(
      (0): PairBilinear()
      (1): PairBilinear()
      (2): PairBilinear()
      (3): PairBilinear()
      (4): PairBilinear()
      (5): PairBilinear()
      (6): PairBilinear()
      (7): PairBilinear()
      (8): PairBilinear()
      (9): PairBilinear()
    )
  )
  (bn1): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=784, out_features=10, bias=True)
)

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [46]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  221882


In [47]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

100%|█████████████████████████████████████████████████| 1200/1200 [01:27<00:00, 13.78it/s]


Epoch: 0,  Loss:0.45773711800575256


100%|███████████████████████████████████████████████████| 200/200 [00:02<00:00, 67.41it/s]


Train Acc:82.77%, Test Acc:85.76%



100%|█████████████████████████████████████████████████| 1200/1200 [01:29<00:00, 13.46it/s]


...

100%|███████████████████████████████████████████████████| 200/200 [00:02<00:00, 67.69it/s]


Train Acc:91.05%, Test Acc:88.54%



 51%|█████████████████████████▌                        | 613/1200 [00:44<00:44, 13.27it/s]../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [45,0,0], thread: [64,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [45,0,0], thread: [65,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [45,0,0], thread: [66,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [45,0,0], thread: [67,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [45,0,0], thread: [68,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/nat

 51%|█████████████████████████▌                        | 614/1200 [00:44<00:42, 13.74it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
### ON GTX-1080

## stats: 20 epochs || Fact+BN+Linear ; lr0.0001 ##_with 3 bilinear layers
### for factor-net: 5grid : 73706-> 100%|██████████| 1200/1200 [00:24<00:00, 48.44it/s]
########### -> Train Acc 90.3367 ; Test Acc 88.06

### for factor-net: 50grid : 5894906-> 100%|██████████| 1200/1200 [00:28<00:00, 42.74it/s]
########### -> Train Acc 99.985 ; Test Acc 85.85

### for factor-net: 10grid : 250106-> 100%|██████████| 1200/1200 [00:24<00:00, 48.11it/s]
########### -> Train Acc 92.17167 ; Test Acc 88.36

In [ ]:
### for factor-net: fact+bn+relu+linear : 5grid : lr 0.0003
####### -> Train Acc 92.42833333333334 ; Test Acc 88.42

### same : factor-net had default of 3 bilinear layers.. changed to log2(input dim)=10 to properly mix all.
#######  -> 100%|██████████| 1200/1200 [00:50<00:00, 23.88it/s]
### facto-net: fact+bn+relu+linear : 5grid  -> params=221882
######## -> Train Acc 95.165 ; Test Acc 89.45

### ordinary net || linear+BN+Linear : lr=0.0003 : params=624074  -> [579.83it/s]
######## -> Train Acc 95.96166666666667 ; Test Acc 89.33
